In [ ]:
# Importing libraries
import numpy as np 
import pandas as pd 
import os
import sys
from pyspark.sql import SparkSession
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

from pyspark import SparkContaext
from pyspark.sql import SparkSession

# Creating a SparkContext instance 
sc = SparkContext()


In [ ]:
# SDSC limits: 128 executors, 250 GB memory
# This condig is for 30 cores, 240 GB memory:
sc = SparkSession.builder \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g")\
    .config('spark.executor.instances', 29) \
    .getOrCreate()